In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# IMPORTING THE REQUIRED LIBRARIES
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, LSTM
from keras.models import Sequential
import warnings
warnings.filterwarnings("ignore")
from numpy.fft import *
%matplotlib inline

In [ ]:
train_1= pd.read_csv("../input/web-traffic-time-series-forecasting/train_1.csv.zip")
train_2= pd.read_csv("../input/web-traffic-time-series-forecasting/train_2.csv.zip")
key_1= pd.read_csv("../input/web-traffic-time-series-forecasting/key_1.csv.zip")
key_2= pd.read_csv("../input/web-traffic-time-series-forecasting/key_2.csv.zip")

In [ ]:
train_1.head(20)

In [ ]:
key_2.head()

In [ ]:
train_1= train_1.fillna(0)
train_2= train_2.fillna

In [ ]:
# FUNCTION FOR CREATING ANOTHER COLUMN IN TRAIN DATASET FOR THE PAGE LANGAUGE
def find_lang(page):
    res= re.search("[a-z][a-z].wikipedia.org", page)
    if res:
        return res[0][0:2]
    return("na")

train_1["lang"]= train_1.Page.map(find_lang)

In [ ]:
sns.countplot(train_1["lang"])
plt.title("Language Distribution")
plt.show()

In [ ]:
lang_set= {}
lang_set["en"]= train_1[train_1.lang=="en"].iloc[:,0:-1]
lang_set["ja"]= train_1[train_1.lang=="ja"].iloc[:,0:-1]
lang_set["de"]= train_1[train_1.lang=="de"].iloc[:,0:-1]
lang_set["na"]= train_1[train_1.lang=="na"].iloc[:,0:-1]
lang_set["fr"]= train_1[train_1.lang=="fr"].iloc[:,0:-1]
lang_set["zh"]= train_1[train_1.lang=="zh"].iloc[:,0:-1]
lang_set["ru"]= train_1[train_1.lang=="ru"].iloc[:,0:-1]
lang_set["es"]= train_1[train_1.lang=="es"].iloc[:,0:-1]

In [ ]:
lang_set["en"]


In [ ]:

sums= {}
for key in lang_set:
    sums[key]= lang_set[key].iloc[:,1:].sum(axis= 0) / lang_set["en"].shape[0]

In [ ]:
sums["en"]


In [ ]:
#plot and see individually traffic on particular language
days= [r for r in range(sums["en"].shape[0])]
fig= plt.figure(1, figsize= [10,10])
plt.ylabel("Views per page")
plt.xlabel("Day")
plt.title("Page in Different Language" )
label= {"en":"English", "ja":"Japanese", "de":"German", "na": "Media",
       "fr": "French", "zh": "Chinese", "ru": "Russian", "es": "Spanish"}
for key in sums:
    plt.plot(days, sums[key], label= label[key])
    
plt.legend()
plt.show()

In [ ]:
def filter_signal(signal, threshold=1e3):
    fourier = rfft(signal)
    frequencies = rfftfreq(signal.size, d=20e-3/signal.size)
    fourier[frequencies > threshold] = 0
    return irfft(fourier)

a= np.array(train_1.iloc[0,1:-1],np.float32)
sc= MinMaxScaler()

plt.figure(figsize= (12,12))
plt.plot(a)
plt.show()

plt.figure(figsize= (12,12))
plt.plot(filter_signal(a))
plt.show()

regressor= Sequential()
regressor.add(LSTM(units= 10, activation="relu",return_sequences=True, input_shape=(None, 1)))
regressor.add(Dense(units = 1))
regressor.compile(optimizer= "adam", loss= "mean_squared_error")

In [ ]:
a= np.array(train_1.iloc[0,1:-1],np.float32)

plt.figure(figsize= (12, 6))
plt.title("Before Fourier Transform")
plt.plot(a)

In [ ]:
plt.figure(figsize= (12,6))
plt.plot(filter_signal(a))
plt.title("After Fourier Transform")

In [ ]:
regressor= Sequential()
regressor.add(LSTM(units= 10, activation="relu",return_sequences=True, input_shape=(None, 1)))
regressor.add(Dense(units = 1))
regressor.compile(optimizer= "adam", loss= "mean_squared_error")

In [ ]:
arr= filter_signal(a)
arr= arr.reshape(-1,1)

# performing standardization of values using min max scaler
arr= sc.fit_transform(arr)
arr= np.reshape(arr, (-1,1,1))

# Train data
arr_X= arr[:400]
arr_y= arr[1:401]
arr_X= np.reshape(arr_X, (-1,1,1))

# Test data
arr_TX= arr[401:-1]
arr_ty= arr[402:]
arr_TX= np.reshape(arr_TX, (-1,1,1))

# Training the model
regressor.fit(arr_X, arr_y, batch_size= 5, epochs= 100, verbose= 0)

# perdicting the value
res= regressor.predict(arr_TX)

# Reshaping the value for plotting purpose
res= res.reshape(148,1)
arr_ty= arr_ty.reshape(148,1)

#plotting the data
plt.plot(res, color="r")
plt.plot(arr_ty, color="b")
plt.show()

In [ ]:
def plot_entry(key, idx):
    data= lang_set[key].iloc[idx,1:]
    fig= plt.figure(1, figsize= (10, 5))
    plt.plot(days, data)
    plt.xlabel("day")
    plt.ylabel("views")
    plt.title(train_1.iloc[lang_set[key].index[idx],0])
    
    plt.show()

In [ ]:
idx= [2, 7, 17, 57, 101, 257, 501, 757, 1117, 1517, 2777]
for i in idx:
    plot_entry("en",i)

In [ ]:
idx= [2, 7, 17, 57, 101, 257, 501, 757, 1117, 1517, 2777]
for i in idx:
    plot_entry("es",i)

For Finding each page individually counts of no of most most in a Page

In [ ]:
# For each language get highest few pages
npages= 5
top_pages= {}
for key in lang_set:
    print(key)
    sum_set= pd.DataFrame(lang_set[key][["Page"]])
    sum_set["Total"]= lang_set[key].sum(axis= 1)
    sum_set = sum_set.sort_values('Total',ascending=False)
    top_pages[key] = sum_set.index[0]
    print(sum_set.head(10))
    print('\n\n')

In [ ]:
for key in top_pages:
    fig = plt.figure(1,figsize=(10,5))
    cols = train_1.columns
    cols = cols[1:-1]
    data = train_1.loc[top_pages[key],cols]
    plt.plot(days,data)
    plt.xlabel('Days')
    plt.ylabel('Views')
    plt.title(train_1.loc[top_pages[key],'Page'])
    plt.show()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import warnings

cols= train_1.columns[1:-1]
for key in top_pages:
    data= np.array(train_1.loc[top_pages[key],cols],"f")
    result= None
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        try:
            arima= ARIMA(data, [2,1,4])
            result= arima.fit(disp= False)
        except:
            try:
                arima= ARIMA(data, [2,1,2])
                result= arima.fit(disp= False)
            except:
                print(train_1.loc[top_pages[key], "Page"])
                print("\tARIMA FAILED")
    pred= result.predict(2,599, typ= "levels")
    x= [i for i in range(600)]
    i=0
    plt.plot(x[2:len(data)],data[2:] ,label='Data')
    plt.plot(x[2:],pred,label='ARIMA Model')
    plt.title(train_1.loc[top_pages[key],'Page'])
    plt.xlabel('Days')
    plt.ylabel('Views')
    plt.legend()
    plt.show()
